In [ ]:

block_size = 10
blockspace = {
    'attnblocksize': 2,
    'local_attn_ctx': 6,
    'block_size': 10,
    'attn_mode': 'fixed',
    'vertsize': 2,
    'n_heads': 4,
    'num_verts': 2,
    'n_ctx': 10
}

print(blockspace(block_size , blockspace))






def blockspace(block_size , cfg):
    n_bctx = block_size // cfg.attnblocksize
    local_attn_ctx = cfg.local_attn_ctx
    block_size = cfg.block_size
    attn_mode = cfg.attn_mode
    vertsize = cfg.vertsize
    n_heads = cfg.n_heads
    num_verts = cfg.num_verts

    layout = np.ones([n_bctx, n_bctx], dtype=np.bool)
    extra_diagonals = None
    block_chunks = None

    if attn_mode in ['all', 'fixed']:
        pass
    elif attn_mode == 'local':
        assert local_attn_ctx % block_size == 0
        extra_diagonals = local_attn_ctx // block_size
    elif attn_mode == 'strided':
        bT_ctx = cfg.n_ctx // local_attn_ctx
        assert bT_ctx % block_size == 0
        block_chunks = bT_ctx // block_size
    else:
        raise ValueError(f'attn mode {attn_mode} invalid')

    if attn_mode == 'fixed':
        assert n_heads % num_verts == 0
        stride = local_attn_ctx // block_size
        assert vertsize <= stride
        assert stride % vertsize == 0

        indices = [i for i in range(stride - 1, -1, -1)]
        indices = np.array(indices).reshape([-1, vertsize])

        if num_verts == 1:
            layout = np.zeros([n_bctx, n_bctx], dtype=np.bool)
            for idx in indices[0]:
                layout[:, idx::stride] = 1
            for q_idx in range(n_bctx):
                row = q_idx // stride
                layout[q_idx, row * stride:(row + 1) * stride] = 1
                layout[q_idx, q_idx + 1:] = 0
        else:
            layouts = []
            indices = indices[:num_verts]
            for h in range(n_heads):
                layout = np.zeros([n_bctx, n_bctx], dtype=np.bool)
                subindices = indices[h % num_verts]
                for idx in subindices:
                    layout[:, idx::stride] = 1
                for q_idx in range(n_bctx):
                    row = q_idx // stride
                    layout[q_idx, row * stride:(row + 1) * stride] = 1
                    layout[q_idx, q_idx + 1:] = 0
                layouts.append(layout)
            layout = np.array(layouts)
    else:
        for q_idx, k_idx in np.ndindex(n_bctx, n_bctx):
            if k_idx > q_idx:
                layout[q_idx, k_idx] = 0
            if extra_diagonals and k_idx + extra_diagonals < q_idx:
                layout[q_idx, k_idx] = 0
            if block_chunks is not None:
                layout[q_idx, k_idx] = 0
                offset = q_idx % block_chunks
                if k_idx + offset >= q_idx and k_idx <= q_idx:
                    layout[q_idx, k_idx] = 1
    return layout

